In [2]:
import json, os, sys
from sentence_transformers import SentenceTransformer, util
from typing import Dict, List, Any
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Optional
from typing import Union, List, Dict


from langgraph.graph.message import add_messages
from pydantic import BaseModel
import ollama

#from langchain.chat_models import ChatOllama
from langchain_ollama import ChatOllama

In [5]:
%pwd

'/Users/danielrubibreton/Documents/CFA'

In [10]:
from transformers import AutoTokenizer, AutoModel

In [11]:
path = "/Users/danielrubibreton/Desktop/PythonStuff/hface/all-MiniLM-L6-v2"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [2]:
with open("cfa2025.json", "r") as file:
    book_json = eval(file.read())

In [3]:
flat_sections = [f"{book} -> {chapter}" for book, chapters in book_json.items() for chapter in chapters]

flat_book = [f"{book} -> {chapter} -> {book_json[book][chapter] }" for book, chapters in book_json.items() for chapter in chapters]

In [4]:
def find_relevant_sections(
    query: str,
    top_k: int = 5,
    score_threshold1: float = 0.5,
    score_threshold2: float = 0.3,
    model_name: str = 'all-MiniLM-L6-v2',
    return_content: bool = False
) -> Union[List[str], Dict[str, str]]:
    """
    If return_content=False: return top_k section titles (e.g. ["Genesis -> 1", ...]).
    If return_content=True: return a dict mapping each section title to its full text.
    """
    model = SentenceTransformer(model_name, device='mps')
    query_emb = model.encode(query, convert_to_tensor=True)

    # 1) Title‐level matching
    sec_embs = model.encode(flat_sections, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, sec_embs)[0]
    hits = sorted(
        [(i, s.item()) for i, s in enumerate(scores) if s.item() >= score_threshold1],
        key=lambda x: x[1], reverse=True
    )[:top_k]

    if not hits:
        # 2) Fallback: book‐level matching
        book_embs = model.encode(flat_book, convert_to_tensor=True)
        scores = util.cos_sim(query_emb, book_embs)[0]
        hits = sorted(
            [(i, s.item()) for i, s in enumerate(scores) if s.item() >= score_threshold2],
            key=lambda x: x[1], reverse=True
        )[: top_k + 2]
        # normalize to just "Book -> Chapter"
        section_keys = []
        for idx, _ in hits:
            book, chap, _ = flat_book[idx].split(" -> ", 2)
            section_keys.append(f"{book} -> {chap}")
    else:
        section_keys = [flat_sections[i] for i, _ in hits]

    if not return_content:
        return section_keys

    # if return_content=True, build the full dict
    return {
        sec: book_json[sec.split(" -> ", 1)[0]][sec.split(" -> ", 1)[1]]
        for sec in section_keys
    }

In [5]:
%%time
res = find_relevant_sections("What is interest rate (or yield)?")

CPU times: user 1.23 s, sys: 592 ms, total: 1.82 s
Wall time: 6 s


In [6]:
res

['Fixed Income -> 2.1.Maturity Structure of Interest Rates',
 'Quantitative Methods -> 2.Interest Rates and Time Value of Money',
 'Fixed Income -> 2.2.Yield-to-Maturity',
 'Quantitative Methods -> 2.1.Determinants of Interest Rates']

In [7]:
class LocalLLM:
    def __init__(
        self,
        model: str = "deepseek-r1:1.5b",
        temperature: float = 0,
        max_tokens: Optional[int] = None,
    ):
        self.model = model
        self.temperature = temperature


    def invoke(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        response = ollama.chat(
            model=self.model,
            messages=messages,
            options={
                "temperature": self.temperature,
                "num_thread": 10,
                "low_vram": False,
            } 
        )
        return response["message"]["content"].split("</think>")[-1]




In [8]:
class GraphState(TypedDict):
    query: str
    context: Optional[str]
    response: Optional[str]
    messages: Annotated[list, add_messages]


In [9]:
def retrieval_node(state: GraphState) -> dict:
    try:
        query = state["messages"][-1].content
    except:
        query = state["query"]

    
    # only pull titles
    section_keys = find_relevant_sections(query)
    # store selected section keys for later full load

    titles = "\n".join(f"{i+1}. {sec}" for i, sec in enumerate(section_keys))
    # ask user to confirm
    prompt = (
        "I found these sections for your query:\n\n"
        f"{titles}\n\n"
        "Are these the sections you want to use? (yes/no)"
    )
    # send as assistant message
    return {
        "prompt": prompt,
        "section_keys": section_keys,   # stash for full load
    }

def full_retrieval_node(state: GraphState) -> dict:
    # load full text only after confirmation
    keys = state["section_keys"]
    full_ctx = []
    for sec in keys:
        book, chap = sec.split(" -> ", 1)
        text = book_json[book][chap]
        full_ctx.append(f"Book & Chapter: {sec}\n{text}")
    context = "\n\n".join(full_ctx)
    return {"context": context}

# def confirm_node(state: GraphState) -> dict:
#     reply = state["messages"][-1].content.strip().lower()
#     if reply in ("yes", "y", "ok", "sure"):
#         # proceed to full load
#         return {"goto": "full_retrieval"}
#     else:
#         # either cancel or re-run retrieval
#         return {"goto": "retrieve", "prompt": "Okay, let me try again. What would you like to search for?"}


In [10]:
def response_node(state: GraphState) -> dict:
    query = state.query
    context = state.context
    
    prompt = (
        "Answer the following question using only the provided context. "
        f"Context:\n{context}\n\n"
        f"Question:\n{query}"
        "Cite your sources like based on the context, if there is no specific info, answer there is no info"
    )

    llm = LocalLLM(model="deepseek-r1:1.5b")
    response_text = llm.invoke(prompt)

    # Append the assistant's response to the messages
    new_message = {"role": "assistant", "content": response_text}
    updated_messages = state["messages"] + [new_message]

    return {"response": response_text, "messages": updated_messages}


In [12]:
graph = (
    StateGraph(GraphState)
    .add_node("retrieve",       retrieval_node)
    .add_node("confirm",        confirm_node)
    .add_node("full_retrieval", full_retrieval_node)
    .add_node("respond",        response_node)
    
    .add_edge(START,            "retrieve")
    .add_edge("retrieve",       "full_retrieval")
    # .add_edge("retrieve",       "confirm")
    # .add_conditional_edges(
    #     "confirm",
    #     path=lambda out: out.get("goto"),
    #     path_map={"full_retrieval":"full_retrieval","retrieve":"retrieve"},
    # )
    .add_edge("full_retrieval", "respond")
    .add_edge("respond",         END)
    .compile()
)

graph.get_graph().print_ascii()


  +-----------+    
  | __start__ |    
  +-----------+    
         *         
         *         
         *         
   +----------+    
   | retrieve |    
   +----------+    
         *         
         *         
         *         
+----------------+ 
| full_retrieval | 
+----------------+ 
         *         
         *         
         *         
    +---------+    
    | respond |    
    +---------+    
         *         
         *         
         *         
    +---------+    
    | __end__ |    
    +---------+    


In [13]:
# assume `graph` is already defined and compiled as in your notebook

while True:
    user_query = input("You: What is the difference between Venture capital and Private Equity?")
    if user_query.strip().lower() == "exit":
        print("Exiting chat. Goodbye!")
        break

    # build the LangGraph input
    state = {"messages": [{"role": "user", "content": user_query}]}
    
    # stream the response tokens as they arrive
    for msg_chunk, meta in graph.stream(
        state,
        stream_mode="messages"
    ):
        print(meta.get("langgraph_node"))
        # only print from your 'respond' node
        if meta.get("langgraph_node") == "respond" and msg_chunk.content:
            print(msg_chunk.content, end="", flush=True)
    print()  # newline after the full response


You: What is the difference between Venture capital and Private Equity? 


KeyError: 'section_keys'

In [16]:
graph_builder = StateGraph(GraphState)
graph_builder.add_node("retrieve", retrieval_node)
graph_builder.add_node("respond", response_node)

graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("respond", END)

graph_builder.add_edge("retrieve", "respond")

graph = graph_builder.compile()

graph.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| retrieve |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | respond |   
 +---------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [308]:
%%time


query = "What is the difference between Venture capital and Private Equity?"

initial_state = {"query": query}

final_state = graph.invoke(initial_state)
print(final_state["response"])



Venture capital and Private Equity (PE) are two distinct investment strategies with significant differences in their approach, focus, and time horizons.

**Venture Capital:**
- **Definition:** Venture capital refers to funds raised by private companies for various purposes such as expansion, acquisition, debt repayment, or financial distress relief.
- **Focus:** Typically involves the early stages of a company's life cycle, especially startups. It is often tied to specific industries and sectors.
- **Investment Timing:** Focuses on short-term needs, such as scaling up a product or entering new markets.
- **Management:** Often managed by individuals, family members, or government entities, not necessarily part of the company's management team.

**Private Equity (PE):**
- **Definition:** Private equity involves investing in companies with less than 20% control. These are usually established firms with significant assets under management.
- **Focus:** Typically involves more stable inve